In [1]:
import os
if not os.path.isdir('Data'):
    import requests

    os.mkdir('Data')

    url = 'https://www.dropbox.com/s/d939ucy3bumt9hb/data_train.csv?dl=1'
    r = requests.get(url, allow_redirects=True)
    open('Data/data_train.csv', 'wb').write(r.content)

    url = 'https://www.dropbox.com/s/6ghexq1uijis473/data_val.csv?dl=1'
    r = requests.get(url, allow_redirects=True)
    open('Data/data_val.csv', 'wb').write(r.content)

    url = 'https://www.dropbox.com/s/py8um99gnjjwcub/data_test.csv?dl=1'
    r = requests.get(url, allow_redirects=True)
    open('Data/data_test.csv', 'wb').write(r.content)

In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [ ]:
pip install image-classifiers==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.3 MB/s 


In [ ]:
from abc import ABC

import os
import random
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np
from shutil import copyfile
import csv
from classification_models.tfkeras import Classifiers
import gc

from abc import ABC
import tensorflow as tf


In [ ]:
def get_data(filename):
    csvfile = open(filename)
    reader = csv.reader(csvfile, delimiter=';')
    next(reader)
    data = []
    for row in reader:
        item = [row[0], row[2:]]
        data.append(item)
    images = np.zeros((len(data), 48, 48, 1), dtype='float32')
    labels = np.zeros((len(data)), dtype='float32')
    labels_full = np.zeros(shape=(len(data), 7), dtype='float32')
    for i in range(len(data)):
        images[i, :, :, :] = np.array(data[i][1]).reshape((48, 48, 1))
        labels[i] = np.array(data[i][0]).astype('float32')
        labels_full[i, int(labels[i])] = 1
    return images, labels_full


def etl_data(path):
    images, labels = get_data(path)
    images = tf.image.resize(images=images, size=(224, 224), method='bilinear').numpy()
    imagesRGB = np.zeros(shape=(images.shape[0], 224, 224, 3), dtype='float32')
    for i in range(images.shape[0]):
        imagesRGB[i, :, :, :] = tf.image.grayscale_to_rgb(tf.convert_to_tensor(images[i, :, :, :])).numpy()
    return imagesRGB, labels

In [ ]:
class cb3(tf.keras.callbacks.Callback):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.reports = []

    def on_epoch_end(self, epoch, logs={}):
        report = tf.keras.metrics.CategoricalAccuracy()(self.y, self.model.predict(self.x)).numpy()
        self.reports.append(report)
        print("Test Accuracy", report)
        print("")
        return

In [ ]:
def augment(images, params):

    y = images

    if params['flip']:
        y = tf.image.flip_left_right(image=y)

    if params['zoom'] > 0 and params['zoom'] < 1:
        y = tf.image.central_crop(image=y,
                                  central_fraction=params['zoom'])
        y = tf.image.resize(images=y,
                            size=[images.shape[1], images.shape[2]],
                            method='bilinear',
                            preserve_aspect_ratio=False)

    if params['shift_h'] != 0 or params['shift_v'] != 0:
        y = tfa.image.translate(images=y,
                                translations=[params['shift_h'], params['shift_v']],
                                interpolation='bilinear',
                                fill_mode='nearest')
    if params['rot'] != 0:
        y = tfa.image.rotate(images=y,
                             angles=params['rot'],
                             interpolation='bilinear',
                             fill_mode='nearest')

    return y


def TTA_Inference(model, x):
    pred_test = model.predict(x)
    zooms = [1]  # 2
    rotations = [0, 0.4, -0.4]  # 5
    shifts_h = [0, 10, -10]  # 3
    shifts_v = [0, 10, -10]  # 3
    flips = [False, True]  # 2

    default_prediction_weight = 3
    count = default_prediction_weight
    predictions = default_prediction_weight*pred_test

    for i1 in range(len(zooms)):
        for i2 in range(len(rotations)):
            for i3 in range(len(shifts_h)):
                for i4 in range(len(shifts_v)):
                    for i5 in range(len(flips)):
                        params = {'zoom': zooms[i1],
                                  'rot': rotations[i2],
                                  'shift_h': shifts_h[i3],
                                  'shift_v': shifts_v[i4],
                                  'flip': flips[i5]}
                        if params['zoom'] < 1 or params['rot'] != 0 or params['shift_h'] != 0 or params['shift_v'] != 0 or params['flip']:

                            count = count + 1
                            d = augment(x, params)
                            preds = model.predict(d, batch_size=128)
                            predictions = predictions + preds
                            del d
                            del preds
                            del params
                            gc.collect()
                            gc.collect()
                            gc.collect()

    Params = [[0.9, 0, 0, 0, False],
              [0.9, 0, 0, 0, True],
              [0.9, 0.15, 0, 0, False],
              [0.9, 0.15, 0, 0, True],
              [0.9, -0.15, 0, 0, False],
              [0.9, -0.15, 0, 0, True]]

    for i in range(len(Params)):
        params = {'zoom': Params[i][0],
                  'rot': Params[i][1],
                  'shift_h': Params[i][2],
                  'shift_v': Params[i][3],
                  'flip': Params[i][4]}
        count = count + 1
        d = augment(x, params)
        preds = model.predict(d, batch_size=128)
        predictions = predictions + preds

        del d
        del preds
        del params
        gc.collect()
        gc.collect()
        gc.collect()

    predictions = predictions / count
    return predictions


def Check_Unique(x):
    lose = 0
    for i in range(x.shape[0]):
        if sum(x[i, :] == x[i, :].max()) > 1:
            lose = lose + 1
    return lose

In [ ]:
import zipfile
if not os.path.isdir('Data_Images'):
  with zipfile.ZipFile("/content/drive/MyDrive/Meta_dataset/Training.zip", 'r') as zip_ref:
        zip_ref.extractall("Data_Images")

In [ ]:
# rm -R /content/Data_Images

In [ ]:
print(tf.__version__)


# resolution^2 deve essere divisibile per num_heads
class LHC_Module(tf.keras.layers.Layer):
    def __init__(self, pool_size, head_emb_dim, num_heads, num_channels, resolution, kernel_size, norm_c, name):
        super(LHC_Module, self).__init__()
        self.pool_size = pool_size
        self.head_emb_dim = head_emb_dim
        self.num_heads = num_heads
        self.num_channels = num_channels
        self.resolution = resolution
        self.kernel_size = kernel_size
        self.norm_c = norm_c

        self.Poolq = tf.keras.layers.AvgPool2D(pool_size=(self.pool_size, self.pool_size),
                                               strides=(1, 1),
                                               padding='same')
        self.Poolk = tf.keras.layers.MaxPool2D(pool_size=(self.pool_size, self.pool_size),
                                               strides=(1, 1),
                                               padding='same')

        self.Wqk = [tf.keras.layers.Dense(units=self.head_emb_dim, activation='linear') for _ in range(self.num_heads)]

        self.Wp = tf.keras.layers.Dense(units=self.num_channels, activation='sigmoid')

        self.Wv = tf.keras.layers.Conv2D(filters=self.num_channels,
                                         kernel_size=self.kernel_size,
                                         strides=(1, 1),
                                         padding='same',
                                         activation='linear')
        self.Poolv = tf.keras.layers.AvgPool2D(pool_size=(3, 3),
                                               strides=(1, 1),
                                               padding='same')

        self.sum = tf.keras.layers.Add()

        self.Name_1_ = 'LHC_1_'+name

    def VectScaledDotProdAttention(self, query, key, value):
        scores = tf.linalg.matmul(query, key, transpose_b=True)  # (batch_size, num_heads, num_channels, num_channels)
        scores_p = tf.math.reduce_mean(scores, axis=3)  # (batch_size, num_heads, num_channels)
        scores_p = self.Wp(scores_p)  # (batch_size, num_heads, num_channels)
        scores_p = tf.expand_dims(scores_p, axis=-1)  # (batch_size, num_heads, num_channels, 1)
        norm_scores = tf.math.divide(scores, tf.math.pow(tf.dtypes.cast(key.shape[3], tf.float32), self.norm_c + scores_p))  # (batch_size, num_heads, num_channels, num_channels)
        weights = tf.nn.softmax(norm_scores, axis=3)  # (batch_size, num_heads, num_channels, num_channels)
        attentions = tf.linalg.matmul(weights, value)  # (batch_size, num_heads, num_channels, head_res_dim)
        return attentions

    def call(self, x):
        batch_size = tf.shape(x)[0]
        num_channels = self.num_channels
        resolution = self.resolution
        head_res_dim = (resolution * resolution) // self.num_heads

        query = x  # (batch_size, resolution, resolution, num_channels)
        query = self.Poolq(query)  # (batch_size, resolution, resolution, num_channels)
        query = tf.reshape(query, shape=(batch_size, resolution * resolution, num_channels))  # (batch_size, resolution^2, num_channels)
        query = tf.transpose(query, perm=[0, 2, 1])  # (batch_size, num_channels, resolution^2)
        query = tf.reshape(query, shape=(batch_size, num_channels, self.num_heads, head_res_dim))  # (batch_size, num_channels, num_heads, head_res_dim)
        query = tf.transpose(query, perm=[0, 2, 1, 3])  # (batch_size, num_heads, num_channels, head_res_dim)
        q = [None] * self.num_heads
        for i in range(self.num_heads):
            q[i] = self.Wqk[i](query[:, i, :, :])  # (batch_size, num_channels, head_emb_dim)
            q[i] = tf.expand_dims(q[i], axis=1)  # (batch_size, 1, num_channels, head_emb_dim)
        query = tf.concat(q, axis=1)  # (batch_size, num_heads, num_channels, head_emb_dim)

        key = x  # (batch_size, resolution, resolution, num_channels)
        key = self.Poolk(key)  # (batch_size, resolution, resolution, num_channels)
        key = tf.reshape(key, shape=(batch_size, resolution * resolution, num_channels))  # (batch_size, resolution^2, num_channels)
        key = tf.transpose(key, perm=[0, 2, 1])  # (batch_size, num_channels, resolution^2)
        key = tf.reshape(key, shape=(batch_size, num_channels, self.num_heads, head_res_dim))  # (batch_size, num_channels, num_heads, head_res_dim)
        key = tf.transpose(key, perm=[0, 2, 1, 3])  # (batch_size, num_heads, num_channels, head_res_dim)
        k = [None] * self.num_heads
        for i in range(self.num_heads):
            k[i] = self.Wqk[i](key[:, i, :, :])  # (batch_size, num_channels, head_emb_dim)
            k[i] = tf.expand_dims(k[i], axis=1)  # (batch_size, 1, num_channels, head_emb_dim)
        key = tf.concat(k, axis=1)  # (batch_size, num_heads, num_channels, head_emb_dim)

        value = self.Wv(x)  # (batch_size, resolution, resolution, num_channels)
        value = self.Poolv(value)  # (batch_size, resolution, resolution, num_channels)
        value = tf.reshape(value, shape=(batch_size, resolution * resolution, num_channels))  # (batch_size, resolution^2, num_channels)
        value = tf.transpose(value, perm=[0, 2, 1])  # (batch_size, num_channels, resolution^2)
        value = tf.reshape(value, shape=(batch_size, num_channels, self.num_heads, head_res_dim))  # (batch_size, num_channels, num_heads, head_res_dim)
        value = tf.transpose(value, perm=[0, 2, 1, 3])  # (batch_size, num_heads, num_channels, head_res_dim)

        attentions = self.VectScaledDotProdAttention(query, key, value)  # (batch_size, num_heads, num_channels, head_res_dim)

        attentions = tf.transpose(attentions, perm=[0, 2, 1, 3])  # (batch_size, num_channels, num_heads, head_res_dim)
        attention = tf.reshape(attentions, shape=(batch_size, num_channels, resolution * resolution))  # (batch_size, num_channels, resolution^2)

        attention = tf.transpose(attention, perm=[0, 2, 1])  # (batch_size, resolution^2, num_channels)
        attention = tf.reshape(attention, shape=(batch_size, resolution, resolution, num_channels))  # (batch_size, resolution, resolution, num_channels)

        out = self.sum([x, attention])  # (batch_size, resolution, resolution, num_channels)

        return out


class LHCResBlockSmall(tf.keras.Model, ABC):
    def __init__(self, filters, kernels, strides, identity, resolution, att_num_channel, num_heads, att_embed_dim, att_kernel_size, pool_size, norm_c, name):
        super(LHCResBlockSmall, self).__init__(name='LHCResBlockSmall')
        self.Identity = identity

        self.bn1 = tf.keras.layers.BatchNormalization(epsilon=2e-05, name=name+"_BN1")
        self.relu1 = tf.keras.layers.Activation(activation='relu', name=name+"_Relu1")
        self.pad1 = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name=name+'_Padding1')
        self.conv1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernels[0], strides=strides[0], padding='valid', activation='linear', use_bias=False, name=name+'_Conv1')

        self.bn2 = tf.keras.layers.BatchNormalization(epsilon=2e-05, name=name+"_BN2")
        self.relu2 = tf.keras.layers.Activation(activation='relu', name=name+"_Relu2")
        self.pad2 = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name=name+'_Padding2')
        self.conv2 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernels[1], strides=strides[1], padding='valid', activation='linear', use_bias=False, name=name+'_Conv2')

        if self.Identity:
            self.convId = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernels[2], strides=strides[2], padding='valid', activation='linear', use_bias=False, name=name+'_ConvId')

        self.LHC_Module = LHC_Module(pool_size=pool_size,
                                     resolution=resolution,
                                     num_channels=att_num_channel,
                                     num_heads=num_heads,
                                     head_emb_dim=att_embed_dim,
                                     kernel_size=att_kernel_size,
                                     norm_c=norm_c,
                                     name=name)

        self.add = tf.keras.layers.Add()

    def call(self, x):
        if self.Identity:
            y = self.bn1(x)
            y = self.relu1(y)
            xb = y
            y = self.pad1(y)
            y = self.conv1(y)

            y = self.bn2(y)
            y = self.relu2(y)
            y = self.pad2(y)
            y = self.conv2(y)

            y2 = self.convId(xb)

            y = self.add([y, y2])

            y = self.LHC_Module(y)

            return y
        else:
            y = self.bn1(x)
            y = self.relu1(y)
            y = self.pad1(y)
            y = self.conv1(y)

            y = self.bn2(y)
            y = self.relu2(y)
            y = self.pad2(y)
            y = self.conv2(y)

            y = self.add([y, x])

            y = self.LHC_Module(y)

            return y

    def import_w(self, layers):
        for i in range(len(layers)):
            for j in range(len(layers[i].weights)):
                self.layers[i].weights[j].assign(layers[i].weights[j])


class LHCResBlockSmall0(tf.keras.Model, ABC):
    def __init__(self, input_shape, resolution, att_num_channel, num_heads, att_embed_dim, att_kernel_size, pool_size, norm_c):
        super(LHCResBlockSmall0, self).__init__(name='LHCResBlockSmall0')

        self.Input = tf.keras.layers.InputLayer(input_shape=input_shape)

        self.bn1 = tf.keras.layers.BatchNormalization(epsilon=2e-05, scale=False, name='Block0_BN1')
        self.pad1 = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)), name='Block0_Padding1')
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='valid', activation='linear', use_bias=False, name='Block0_Conv1')
        self.bn2 = tf.keras.layers.BatchNormalization(epsilon=2e-05, name='Block0_BN2')
        self.relu1 = tf.keras.layers.Activation(activation='relu', name='Block0_Relu1')
        self.pad2 = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name='Block0_Padding2')
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), name='Block0_MaxPool1')
        self.LHC_Module = LHC_Module(pool_size=pool_size,
                                     resolution=resolution,
                                     num_channels=att_num_channel,
                                     num_heads=num_heads,
                                     head_emb_dim=att_embed_dim,
                                     kernel_size=att_kernel_size,
                                     norm_c=norm_c,
                                     name='Module_0')

    def call(self, x):
        x1 = self.Input(x)
        x1 = self.bn1(x1)
        x1 = self.pad1(x1)
        x1 = self.conv1(x1)
        x1 = self.bn2(x1)
        x1 = self.relu1(x1)
        x1 = self.pad2(x1)
        x1 = self.pool1(x1)

        x1 = self.LHC_Module(x1)
        return x1

    def import_w(self, layers):
        for i in range(len(layers)):
            for j in range(len(layers[i].weights)):
                self.layers[i].weights[j].assign(layers[i].weights[j])


class ResBlockSmall(tf.keras.Model, ABC):
    def __init__(self, filters, kernels, strides, identity, name):
        super(ResBlockSmall, self).__init__(name='ResBlockSmall')
        self.Identity = identity

        self.bn1 = tf.keras.layers.BatchNormalization(epsilon=2e-05, name=name+"_BN1")
        self.relu1 = tf.keras.layers.Activation(activation='relu', name=name+"_Relu1")
        self.pad1 = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name=name+'_Padding1')
        self.conv1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernels[0], strides=strides[0], padding='valid', activation='linear', use_bias=False, name=name+'_Conv1')

        self.bn2 = tf.keras.layers.BatchNormalization(epsilon=2e-05, name=name+"_BN2")
        self.relu2 = tf.keras.layers.Activation(activation='relu', name=name+"_Relu2")
        self.pad2 = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name=name+'_Padding2')
        self.conv2 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernels[1], strides=strides[1], padding='valid', activation='linear', use_bias=False, name=name+'_Conv2')

        if self.Identity:
            self.convId = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernels[2], strides=strides[2], padding='valid', activation='linear', use_bias=False, name=name+'_ConvId')

        self.add = tf.keras.layers.Add()

    def call(self, x):
        if self.Identity:
            y = self.bn1(x)
            y = self.relu1(y)
            xb = y
            y = self.pad1(y)
            y = self.conv1(y)

            y = self.bn2(y)
            y = self.relu2(y)
            y = self.pad2(y)
            y = self.conv2(y)

            y2 = self.convId(xb)

            y = self.add([y, y2])
            return y
        else:
            y = self.bn1(x)
            y = self.relu1(y)
            y = self.pad1(y)
            y = self.conv1(y)

            y = self.bn2(y)
            y = self.relu2(y)
            y = self.pad2(y)
            y = self.conv2(y)

            y = self.add([y, x])
            return y

    def import_w(self, layers):
        for i in range(len(layers)):
            for j in range(len(layers[i].weights)):
                self.layers[i].weights[j].assign(layers[i].weights[j])


class LHC_ResNet34(tf.keras.Model, ABC):
    def __init__(self, input_shape, num_classes, att_params):
        super(LHC_ResNet34, self).__init__(name='LHC_ResNet34')

        self.Input = tf.keras.layers.InputLayer(input_shape=input_shape)

        self.conv1 = LHCResBlockSmall0(input_shape=input_shape,
                                       resolution=56,
                                       att_num_channel=64,
                                       num_heads=att_params['num_heads'][0],
                                       att_embed_dim=att_params['att_embed_dim'][0],
                                       att_kernel_size=att_params['kernel_size'][0],
                                       pool_size=att_params['pool_size'][0],
                                       norm_c=att_params['norm_c'][0])

        self.conv2_1 = ResBlockSmall(filters=64, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)), identity=True, name='stage1_unit1')
        self.conv2_2 = ResBlockSmall(filters=64, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)), identity=False, name='stage1_unit2')
        self.conv2_3 = LHCResBlockSmall(filters=64,
                                        kernels=((3, 3), (3, 3), (1, 1)),
                                        strides=((1, 1), (1, 1), (1, 1)),
                                        identity=False,
                                        resolution=56,
                                        att_num_channel=64,
                                        num_heads=att_params['num_heads'][1],
                                        att_embed_dim=att_params['att_embed_dim'][1],
                                        att_kernel_size=att_params['kernel_size'][1],
                                        pool_size=att_params['pool_size'][1],
                                        norm_c=att_params['norm_c'][1],
                                        name='stage1_unit3')

        self.conv3_1 = ResBlockSmall(filters=128, kernels=((3, 3), (3, 3), (1, 1)), strides=((2, 2), (1, 1), (2, 2)), identity=True, name='stage2_unit1')
        self.conv3_2 = ResBlockSmall(filters=128, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)), identity=False, name='stage2_unit2')
        self.conv3_3 = ResBlockSmall(filters=128, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)), identity=False, name='stage2_unit3')
        self.conv3_4 = LHCResBlockSmall(filters=128,
                                        kernels=((3, 3), (3, 3), (1, 1)),
                                        strides=((1, 1), (1, 1), (1, 1)),
                                        identity=False,
                                        resolution=28,
                                        att_num_channel=128,
                                        num_heads=att_params['num_heads'][2],
                                        att_embed_dim=att_params['att_embed_dim'][2],
                                        att_kernel_size=att_params['kernel_size'][2],
                                        pool_size=att_params['pool_size'][2],
                                        norm_c=att_params['norm_c'][2],
                                        name='stage2_unit4')

        self.conv4_1 = ResBlockSmall(filters=256, kernels=((3, 3), (3, 3), (1, 1)), strides=((2, 2), (1, 1), (2, 2)), identity=True, name='stage3_unit1')
        self.conv4_2 = ResBlockSmall(filters=256, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)), identity=False, name='stage3_unit2')
        self.conv4_3 = ResBlockSmall(filters=256, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)), identity=False, name='stage3_unit3')
        self.conv4_4 = ResBlockSmall(filters=256, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)), identity=False, name='stage3_unit4')
        self.conv4_5 = ResBlockSmall(filters=256, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)), identity=False, name='stage3_unit5')
        self.conv4_6 = LHCResBlockSmall(filters=256,
                                        kernels=((3, 3), (3, 3), (1, 1)),
                                        strides=((1, 1), (1, 1), (1, 1)),
                                        identity=False,
                                        resolution=14,
                                        att_num_channel=256,
                                        num_heads=att_params['num_heads'][3],
                                        att_embed_dim=att_params['att_embed_dim'][3],
                                        att_kernel_size=att_params['kernel_size'][3],
                                        pool_size=att_params['pool_size'][3],
                                        norm_c=att_params['norm_c'][3],
                                        name='stage3_unit6')

        self.conv5_1 = ResBlockSmall(filters=512, kernels=((3, 3), (3, 3), (1, 1)), strides=((2, 2), (1, 1), (2, 2)),  identity=True, name='stage4_unit1')
        self.conv5_2 = ResBlockSmall(filters=512, kernels=((3, 3), (3, 3), (1, 1)), strides=((1, 1), (1, 1), (1, 1)),  identity=False, name='stage4_unit2')
        self.conv5_3 = LHCResBlockSmall(filters=512,
                                        kernels=((3, 3), (3, 3), (1, 1)),
                                        strides=((1, 1), (1, 1), (1, 1)),
                                        identity=False,
                                        resolution=7,
                                        att_num_channel=512,
                                        num_heads=att_params['num_heads'][4],
                                        att_embed_dim=att_params['att_embed_dim'][4],
                                        att_kernel_size=att_params['kernel_size'][4],
                                        pool_size=att_params['pool_size'][4],
                                        norm_c=att_params['norm_c'][4],
                                        name='stage4_unit3')

        self.bn = tf.keras.layers.BatchNormalization(epsilon=2e-05, name='bn')
        self.relu = tf.keras.layers.Activation(activation='relu', name='relu')

        self.pool = tf.keras.layers.GlobalAveragePooling2D()

        self.fc1 = tf.keras.layers.Dense(units=4096, activation='relu')
        self.dp1 = tf.keras.layers.Dropout(0.4)
        self.fc2 = tf.keras.layers.Dense(units=1024, activation='relu')
        self.dp2 = tf.keras.layers.Dropout(0.4)
        self.fc3 = tf.keras.layers.Dense(units=num_classes, activation='softmax')

    def import_w(self, model):
        self.conv1.import_w(model.layers[1].layers[0:8])

        self.conv2_1.import_w(model.layers[1].layers[8:18-1])
        self.conv2_2.import_w(model.layers[1].layers[18:27-1])
        self.conv2_3.import_w(model.layers[1].layers[27:36-1])

        self.conv3_1.import_w(model.layers[1].layers[36:46-1])
        self.conv3_2.import_w(model.layers[1].layers[46:55-1])
        self.conv3_3.import_w(model.layers[1].layers[55:64-1])
        self.conv3_4.import_w(model.layers[1].layers[64:73-1])

        self.conv4_1.import_w(model.layers[1].layers[73:83-1])
        self.conv4_2.import_w(model.layers[1].layers[83:92-1])
        self.conv4_3.import_w(model.layers[1].layers[92:101-1])
        self.conv4_4.import_w(model.layers[1].layers[101:110-1])
        self.conv4_5.import_w(model.layers[1].layers[110:119-1])
        self.conv4_6.import_w(model.layers[1].layers[119:128-1])

        self.conv5_1.import_w(model.layers[1].layers[128:138-1])
        self.conv5_2.import_w(model.layers[1].layers[138:147-1])
        self.conv5_3.import_w(model.layers[1].layers[147:156-1])

        for i in range(len(self.bn.weights)):
            self.bn.weights[i].assign(model.layers[1].layers[156].weights[i])

        for i in range(len(self.relu.weights)):
            self.relu.weights[i].assign(model.layers[1].layers[157].weights[i])

        for i in range(len(self.pool.weights)):
            self.pool.weights[i].assign(model.layers[2].weights[i])

        for i in range(len(self.fc1.weights)):
            self.fc1.weights[i].assign(model.layers[3].weights[i])

        for i in range(len(self.dp1.weights)):
            self.dp1.weights[i].assign(model.layers[4].weights[i])

        for i in range(len(self.fc2.weights)):
            self.fc2.weights[i].assign(model.layers[5].weights[i])

        for i in range(len(self.dp2.weights)):
            self.dp2.weights[i].assign(model.layers[6].weights[i])

        for i in range(len(self.fc3.weights)):
            self.fc3.weights[i].assign(model.layers[7].weights[i])

    def call(self, x):
        x = self.Input(x)
        x = self.conv1(x)
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.conv2_3(x)
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.conv3_4(x)
        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        x = self.conv4_4(x)
        x = self.conv4_5(x)
        x = self.conv4_6(x)
        x = self.conv5_1(x)
        x = self.conv5_2(x)
        x = self.conv5_3(x)

        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.fc1(x)
        x = self.dp1(x)
        x = self.fc2(x)
        x = self.dp2(x)
        x = self.fc3(x)
        return x

2.9.2


In [ ]:
if not os.path.isdir('Models'):
    os.mkdir('Models')

import json
with open('/content/Settings/Training_Settings.json') as json_file1:
    training_settings = json.load(json_file1)
with open('/content/Settings/LHC_Settings.json') as json_file2:
    lhc_settings = json.load(json_file2)

Params = lhc_settings

del lhc_settings

seed = training_settings['seed']

os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(seed)
random.seed(seed)
np.random.seed(seed)

path_val = "/content/Data/data_val.csv"
validation_imagesRGB, validation_labels = etl_data(path_val)

Categories = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']




ResNet34, preprocess_input = Classifiers.get('resnet34')
resnet_model = ResNet34(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = resnet_model.input
y = preprocess_input(x)
y = resnet_model(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(units=4096, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=3))(y)
y = tf.keras.layers.Dropout(rate=0.4)(y)
y = tf.keras.layers.Dense(units=1024, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=4))(y)
y = tf.keras.layers.Dropout(rate=0.4)(y)
y = tf.keras.layers.Dense(units=7, activation='softmax')(y)
base_model = tf.keras.Model(inputs=x, outputs=y)

model = LHC_ResNet34(input_shape=(224, 224, 3), num_classes=7, att_params=Params)
x0 = np.ones(shape=(10, 224, 224, 3), dtype='float32')
y0 = model(x0)
model.import_w(base_model)

del x
del y
del base_model
del ResNet34
del preprocess_input
del resnet_model
gc.collect()

85521592/85521592 [==============================] - 1s 0us/step


5123

In [ ]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=None,
    rotation_range=training_settings['rotation_range'][0],
    width_shift_range=training_settings['width_shift_range'][0],
    height_shift_range=training_settings['height_shift_range'][0],
    shear_range=training_settings['shear_range'][0],
    zoom_range=training_settings['zoom_range'][0],
    horizontal_flip=training_settings['horizontal_flip'][0]
)

train_generator = train_data_gen.flow_from_directory(
    directory='/content/Data_Images/Training',
    target_size=(224, 224),
    color_mode='rgb',
    classes=Categories,
    class_mode='categorical',
    batch_size=training_settings['batch_size'][0],
    seed=seed+1
)

opt = tf.keras.optimizers.Adam(learning_rate=training_settings['learning_rate'][0],
                               beta_1=0.9,
                               beta_2=0.999,
                               epsilon=1e-07,
                               amsgrad=False,
                               name='Adam')

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', 
                                             patience=training_settings['patience'][0], 
                                             restore_best_weights=True)
callback2 = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Meta_dataset/Models', 
                                               monitor='val_categorical_accuracy', 
                                               verbose=1, 
                                               save_best_only=True)

for i in range(0,15):
  history = model.fit(
      train_generator,
      epochs=300,
      verbose=1,
      callbacks=[callback1, callback2],
      validation_data=(validation_imagesRGB, validation_labels)
  )
  model.save_weights("/content/drive/MyDrive/Meta_dataset/h5_model1")

Found 28709 images belonging to 7 classes.
Epoch 1/300
599/599 [==============================] - ETA: 0s - loss: 1.8086 - categorical_accuracy: 0.2516

In [ ]:
# #  2.2
# train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=None,
#     rotation_range=training_settings['rotation_range'][1],
#     width_shift_range=training_settings['width_shift_range'][1],
#     height_shift_range=training_settings['height_shift_range'][1],
#     shear_range=training_settings['shear_range'][1],
#     zoom_range=training_settings['zoom_range'][1],
#     horizontal_flip=training_settings['horizontal_flip'][1]
# )


# train_generator = train_data_gen.flow_from_directory(
#     directory='/content/Data_Images/Training',
#     target_size=(224, 224),
#     color_mode='rgb',
#     classes=Categories,
#     class_mode='categorical',
#     batch_size=training_settings['batch_size'][1],
#     seed=seed+1
# )

In [ ]:
# opt = tf.keras.optimizers.SGD(learning_rate=training_settings['learning_rate'][1])

# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=training_settings['patience'][1], restore_best_weights=True)
# callback2 = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Meta_dataset/Models', monitor='val_categorical_accuracy', verbose=1, save_best_only=True)

# history2 = model.fit(
#     train_generator,
#     epochs=300,
#     verbose=1,
#     callbacks=[callback1, callback2],
#     validation_data=(validation_imagesRGB, validation_labels)
# )
# model.save_weights("/content/drive/MyDrive/Meta_dataset/h5_model2")

In [ ]:
# #  3.2
# train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=None,
# )


# train_generator = train_data_gen.flow_from_directory(
#     directory='/content/Data_Images/Training',
#     target_size=(224, 224),
#     color_mode='rgb',
#     classes=Categories,
#     class_mode='categorical',
#     batch_size=training_settings['batch_size'][2],
#     seed=seed+2
# )

In [ ]:
# opt = tf.keras.optimizers.SGD(learning_rate=training_settings['learning_rate'][2])
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=training_settings['patience'][2], restore_best_weights=True)
# callback2 = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Meta_dataset/Models', monitor='val_categorical_accuracy', verbose=1, save_best_only=True)


# history3 = model.fit(
#     train_generator,
#     epochs=300,
#     verbose=1,
#     callbacks=[callback1, callback2],
#     validation_data=(validation_imagesRGB, validation_labels)
# )
# model.save_weights("/content/drive/MyDrive/Meta_dataset/h5_model3")

In [ ]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'

tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)



# DATA IMPORT
Categories = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']

full_train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=None
)

full_train_generator = full_train_data_gen.flow_from_directory(
    directory='/content/Data_Images/Training',
    target_size=(224, 224),
    color_mode='rgb',
    classes=Categories,
    class_mode='categorical',
    batch_size=28709,
    shuffle=False
)

h = full_train_generator.next()
training_imagesRGB = h[0]
training_labels = h[1]

del h
gc.collect()

path_val = "/content/Data/data_val.csv"
path_test = "/content/Data/data_test.csv"
validation_imagesRGB, validation_labels = etl_data(path_val)
testing_imagesRGB, testing_labels = etl_data(path_test)







# MODEL IMPORT
Params = {'num_heads': [8, 8, 7, 7, 1],
          'att_embed_dim': [196, 196, 56, 14, 25],
          'kernel_size': [3, 3, 3, 3, 3],
          'pool_size': [3, 3, 3, 3, 3],
          'norm_c': [1, 1, 1, 1, 1]}
model = LHC_ResNet34(input_shape=(224, 224, 3), num_classes=7, att_params=Params)
x0 = np.ones(shape=(10, 224, 224, 3), dtype='float32')
y0 = model(x0)
model.load_weights('/content/drive/MyDrive/Meta_dataset/Models')



#METRICS
print("LHC_Net Perf:")
pred_train = model.predict(training_imagesRGB)
loss_train = tf.keras.losses.CategoricalCrossentropy()(training_labels, pred_train).numpy()
perf_train = tf.keras.metrics.CategoricalAccuracy(dtype='float64')(training_labels, pred_train).numpy()
print('Train Loss: ', '%.17f' % loss_train)
print('Train Perf: ', '%.17f' % perf_train)
pred_val = model.predict(validation_imagesRGB)
perf_val = tf.keras.metrics.CategoricalAccuracy(dtype='float64')(validation_labels, pred_val).numpy()
print('Val Perf: ', '%.17f' % perf_val)
pred_test = model.predict(testing_imagesRGB)
perf_test = tf.keras.metrics.CategoricalAccuracy(dtype='float64')(testing_labels, pred_test).numpy()
print('Test Perf: ', '%.17f' % perf_test)
pred_test_uniqueness = Check_Unique(pred_test)
print('Test Pred Repeated: ', pred_test_uniqueness)




# TTA
tf.config.run_functions_eagerly(True)
tta_pred_test = TTA_Inference(model, testing_imagesRGB)
tf.config.run_functions_eagerly(False)







# METRICS TTA
tta_perf_test = tf.keras.metrics.CategoricalAccuracy(dtype='float64')(testing_labels, tta_pred_test).numpy()
print('TTA Test Perf: ', '%.17f' % tta_perf_test)
print("")
tta_pred_test_uniqueness = Check_Unique(tta_pred_test)
print('TTA Test Pred Repeated: ', tta_pred_test_uniqueness)
print("")
print("")
print("")